# DeepChem

- [Installation](https://github.com/deepchem/deepchem#installation)
- [Tutorial](https://deepchem.readthedocs.io/en/latest/get_started/tutorials.html)
- [Sample Notebooks](https://github.com/deepchem/deepchem/tree/master/examples/tutorials) 

In [15]:
import pandas as pd
import numpy as np

import deepchem as dc
from deepchem.feat.mol_graphs import ConvMol

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Parameters , Settings

In [47]:
data_filepath = './data/ESOL.csv' 
model_dir = './result'   # to save fitted model

In [3]:
# Initialze the metrics
# https://deepchem.readthedocs.io/en/latest/api_reference/metrics.html
metric_r2 = dc.metrics.Metric(dc.metrics.r2_score)
metric_mse = dc.metrics.Metric(dc.metrics.mean_squared_error)
metrics = [metric_r2, metric_mse]

# Load Data 

In [4]:
data = pd.read_csv(data_filepath)   
data.shape

(1128, 2)

In [38]:
smiles = data['smiles']   # should be 1D
y = data['measured log solubility in mols per litre']   # can be 1D or 2D

# Make Dataset for DeepChem

In [7]:
# Feature engineering
# https://deepchem.readthedocs.io/en/latest/api_reference/featurizers.html
featurizer = dc.feat.CircularFingerprint(radius=2, size=1024)

In [9]:
# "smiles" can be Pandas Series or List. This method also accepts RDKit "mol" objects as input.
# We don't apply feature selection here. This tutorial is an introduction of DeepChem. 
features = featurizer.featurize(smiles)   # array with shape : [No. of Samples, Size of Features]
features.shape

(1128, 1024)

In [10]:
# "NumpyDataset" class stores datasets in memory. This works fine for smaller datasets, but is less convenient for larger datasets. 
# For large datasets, use "DiskDataset" which writes dataset to a folder. 
# https://deepchem.readthedocs.io/en/latest/api_reference/data.html
# https://github.com/deepchem/deepchem/blob/master/examples/tutorials/02_Working_With_Datasets.ipynb
dataset = dc.data.NumpyDataset(X=features, y=np.array(y))
print(dataset)

<NumpyDataset X.shape: (1128, 1024), y.shape: (1128,), w.shape: (1128,), task_names: [0]>


# Split Dataset

In [11]:
# "random splitting" sometimes overestimates model’s performance, especially for small data or imbalance data. 
# The dc.splits provides more methods and algorithms to evaluate the model’s performance appropriately, like cross validation or splitting using molecular scaffolds.
# https://deepchem.readthedocs.io/en/latest/api_reference/splitters.html

splitter = dc.splits.RandomSplitter()

train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset=dataset, 
            frac_train=0.7, frac_valid=0.15, frac_test=0.15)

In [12]:
print(test_dataset)

<NumpyDataset X.shape: (170, 1024), y.shape: (170,), w.shape: (170,), ids: [582 600 762 ... 578 106 4], task_names: [0]>


# Scikit-Learn Model

In [13]:
rf = RandomForestRegressor()

model = dc.models.SklearnModel(model=rf)

# model training
model.fit(train_dataset)

valid_preds = model.predict(valid_dataset)
test_preds = model.predict(test_dataset)

In [14]:
# Evaluate the model
print("Training set score:", model.evaluate(train_dataset, metrics))
print("Validation set score:", model.evaluate(valid_dataset, metrics))
print("Test set score:", model.evaluate(test_dataset, metrics))

Training set score: {'r2_score': 0.9413169878770975, 'mean_squared_error': 0.2739201150126245}
Validation set score: {'r2_score': 0.6101626037769429, 'mean_squared_error': 1.4362241449299051}
Test set score: {'r2_score': 0.6423432030476054, 'mean_squared_error': 1.3168607308163904}


In [16]:
# Since we have predicted values, we can use scikit-learn methods to calculate metrics as usual.
y_test = test_dataset.y

print("Test dataset (R2) : ", r2_score(y_test, test_preds))
print("Test dataset (RMSE) : ", np.sqrt(mean_squared_error(y_test, test_preds)))

Test dataset (R2) :  0.6423432030476054
Test dataset (RMSE) :  1.1475455245071502


# FNN Model (Fully-connected Neural Networks)

In [17]:
# Multitask regressor (fully connected network)
# https://deepchem.readthedocs.io/en/latest/api_reference/models.html#multitaskregressor
# layer_sizes : No. of nodes in every layer. In this example, a single hidden layer is used
model = dc.models.MultitaskRegressor(n_tasks=1, n_features=1024, layer_sizes=[500])

loss_avg = model.fit(train_dataset, nb_epoch=50)
loss_avg

0.7227210235595704

In [18]:
print('training set score:', model.evaluate(train_dataset, metrics))
print('validation set score:', model.evaluate(valid_dataset, metrics))
print('test set score:', model.evaluate(test_dataset, metrics))

training set score: {'r2_score': 0.9013138821838904, 'mean_squared_error': 0.4606463057097943}
validation set score: {'r2_score': 0.508651307494114, 'mean_squared_error': 1.8102082114074893}
test set score: {'r2_score': 0.502637112555131, 'mean_squared_error': 1.8312462143108037}


In [19]:
valid_preds = model.predict(valid_dataset)
test_preds = model.predict(test_dataset)

print("Test dataset (R2) : ", r2_score(y_test, test_preds[:,:,0]))
print("Test dataset (RMSE) : ", np.sqrt(mean_squared_error(y_test, test_preds[:,:,0])))

Test dataset (R2) :  0.502637112555131
Test dataset (RMSE) :  1.3532354615183582


# Graph Convolution Featurizers


DeepChem supports lots of different graph based models. Some of them require molecules to be featurized in slightly different ways. 

Because of this, there are two other featurizers called "ConvMolFeaturizer", "WeaveFeaturizer" and "MolGraphConvFeaturizer". 

They each convert molecules into a different type of Python object that is used by particular models. 

When using any graph based model, just check the documentation to see what featurizer you need to use with it.

[Model Cheatsheet](https://deepchem.readthedocs.io/en/latest/api_reference/models.html#model-cheatsheet)

### ConvMolFeaturizer

Duvenaud graph convolutions, Can be used with Keras models

https://deepchem.readthedocs.io/en/latest/api_reference/featurizers.html#convmolfeaturizer

In [32]:
sample_smiles = ["C", "CCC", "C1=CC=CN=C1"]
featurizer = dc.feat.ConvMolFeaturizer()
features = featurizer.featurize(sample_smiles)   # numpy array, it returns a graph object for every molecule
type(features[0])   # deepchem.feat.mol_graphs.ConvMol

deepchem.feat.mol_graphs.ConvMol

In [33]:
# Make a dataset
sample_dataset = dc.data.NumpyDataset(X=features)   # only X is enough
print(sample_dataset)

<NumpyDataset X.shape: (3,), y.shape: (3, 1), w.shape: (3, 1), ids: [0 1 2], task_names: [0]>


In [34]:
# Iterate over dataset
# https://deepchem.readthedocs.io/en/latest/api_reference/data.html#deepchem.data.NumpyDataset.itersamples
inputs = []
for x, y, w, id in sample_dataset.itersamples():    
    print(x, y, w, id)
    inputs_ = [x.get_atom_features(), x.deg_slice, np.array(x.membership)]  
    for i in range(1, len(x.get_deg_adjacency_lists())):
        inputs_.append(x.get_deg_adjacency_lists()[i]) 
    inputs.append(inputs_)

<deepchem.feat.mol_graphs.ConvMol object at 0x7fb672579850> [0.] [0.] 0
<deepchem.feat.mol_graphs.ConvMol object at 0x7fb6725d4750> [0.] [0.] 1
<deepchem.feat.mol_graphs.ConvMol object at 0x7fb6725d4b10> [0.] [0.] 2


- Discussion about graph features:

From the above code, we get graph features as numeric values. For every molecule, it is a List containing 13 numpy arrays.
However, the size of arrays is different for every molecule (because No. of atoms is different).
Therefore, it is not possible to concatenate features of all molecules altogether and make a DataFrame. Because DataFrame requires the same No. of columns for every row (which is a molecule in this case).

- Note:

When you use graph representation of a molecule, every atom has a feature vector. Since the No. of atoms is different in different molecules, the size of graph features is also different. 

### MolGraphConvFeaturizer

General graph convolution networks for molecules, Can be used with PyTorch models

 https://deepchem.readthedocs.io/en/latest/api_reference/featurizers.html#molgraphconvfeaturizer

In [36]:
sample_smiles = ["CCC", "C1=CC=CN=C1"]
featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
features = featurizer.featurize(sample_smiles)   # it returns a graph object for every molecule
type(features[0])   # deepchem.feat.graph_data.GraphData

deepchem.feat.graph_data.GraphData

In [37]:
print("No. of node features : ", features[0].num_node_features)   # 30
print("No. of edge features : ", features[0].num_edge_features)  # 11

No. of node features :  30
No. of edge features :  11


# DeepChem datasets

Look deeper into DeepChem datasets (inherited from MoleculeNet)

We load one of the datasets (for regression task) and apply 'GraphConv' featurizer.

In [40]:
# log solubility in mols per litre
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets

print(f'tasks : {tasks} \n')
print(test_dataset)   # DiskDataset

tasks : ['measured log solubility in mols per litre'] 

<DiskDataset X.shape: (113,), y.shape: (113, 1), w.shape: (113, 1), ids: ['c1cc2ccc3cccc4ccc(c1)c2c34' 'Cc1cc(=O)[nH]c(=S)[nH]1'
 'Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4 ' ...
 'c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43' 'Cc1occc1C(=O)Nc2ccccc2'
 'OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)C(O)C3O '], task_names: ['measured log solubility in mols per litre']>


In [41]:
# Convert into DataFrame
test_dataset_df = test_dataset.to_dataframe()   # DataFrame
test_dataset_df.head(3)

,X,y,w,ids
0,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-1.601145,1.0,c1cc2ccc3cccc4ccc(c1)c2c34
1,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.208483,1.0,Cc1cc(=O)[nH]c(=S)[nH]1
2,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-0.016027,1.0,Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4


#### What is transformers?

https://deepchem.readthedocs.io/en/latest/api_reference/transformers.html#

It includes methods to apply normalization and standard scaling on datasets. DeepChem datasets might have been transformed.  

In [42]:
# Check transformer
transformers[0]   # there is one transformer object (normalization)
transformers[0].transform_X   # False

False

In [43]:
transformers[0].transform_y   # True --> When we calculate metrics, we need to pass transformers to inverse transform "y"

True

In [44]:
# Check data
y_train = train_dataset.y
print(f'mean : {np.mean(y_train)}')   # 0
print(f'mean : {np.std(y_train)}')    # 1

mean : -3.7811586825371185e-16
mean : 0.9999999999999994


In [45]:
x_train = train_dataset.X   # array of graph objects
type(x_train[0])

deepchem.feat.mol_graphs.ConvMol

# Train Graph Convolution Model on DeepChem Dataset

In [48]:
# https://deepchem.readthedocs.io/en/latest/api_reference/models.html#graphconvmodel
# This Class uses Keras models
model = dc.models.GraphConvModel(n_tasks=len(tasks),       # No. of tasks 
                                 graph_conv_layers=[64],   # Width of channels for the Graph Convolution Layers
                                 dense_layer_size=128,     # Width of channels for Atom Level Dense Layer before GraphPool
                                 dropout=0.2,              # Dropout probablity to use for each layer. The length of this list should equal len(graph_conv_layers)+1 (one value for each convolution layer, and one for the dense layer). Alternatively this may be a single value instead of a list, in which case the same value is used for every layer.
                                 mode='regression',        # Either “classification” or “regression”
                                 batch_size=100,           # Batch size for training and evaluating
                                 model_dir=model_dir,      # Directory on disk where the model will be stored. If this is None, a temporary directory is created.
                                 learning_rate=0.001
                                )

loss_avg = model.fit(train_dataset, nb_epoch=50)
loss_avg

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool/Reshape_14:0", shape=(363,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool/Reshape_13:0", shape=(363, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool/Reshape_17:0", shape=(988,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_

0.16439361572265626

In [49]:
model.model.summary()   # must be after "model.fit"

Model: "private__graph_conv_keras_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
graph_conv (GraphConv)       multiple                  102144    
_________________________________________________________________
batch_normalization (BatchNo multiple                  256       
_________________________________________________________________
batch_normalization_1 (Batch multiple                  512       
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
graph_pool (GraphPool)       multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multip

In [50]:
print("Training set score:", model.evaluate(train_dataset, metrics, transformers))
print("Validation set score:", model.evaluate(valid_dataset, metrics, transformers))
print("Test set score:", model.evaluate(test_dataset, metrics, transformers))

# What is "transformers"?
# These transformations must have been applied to dataset previously. The dataset will be untransformed for metric evaluation.
# Transformers for evaluation. This argument is needed since train_dataset and valid_dataset may have been transformed for learning
# and need the transform to be inverted before the metric can be evaluated on a model (inverse transform).

Training set score: {'r2_score': 0.7832451199443394, 'mean_squared_error': 0.9258356402264704}
Validation set score: {'r2_score': 0.5162568494802122, 'mean_squared_error': 1.8904103877036327}
Test set score: {'r2_score': 0.49130788982575313, 'mean_squared_error': 2.2854767038117947}


### Prediction

In [51]:
# input to the "predict" is "dataset"
y_pred = model.predict(test_dataset)   # numpy array [n, 1]

y_test = test_dataset.y

r2_score(y_test, y_pred)

0.49130788982575313